In [1]:
#Importing Required Libraries

import pandas as pd
from isoweek import Week
from datetime import datetime
from datetime import timedelta
import numpy as np
from pandasql import sqldf
pysqldf = lambda q: sqldf(q, globals())
import matplotlib.pyplot as plt
import plotly
import plotly.plotly as py
from plotly.graph_objs import *
import plotly.figure_factory as ff
import seaborn as sns
%matplotlib inline

Using Plotly for Visulization

Also got the file with list US state and Regions from following website

http://researchertools.blogspot.com/2012/09/excel-file-with-us-states-abbreviations.html
    

In [2]:
#Define Regions
plotly.tools.set_credentials_file(username='srikumar.venkatraman', api_key='eM0Z8xYdzbAy43zQrexM')
plotly.offline.init_notebook_mode(connected=True)
regions  = pd.read_excel('US STATES REGIONS SUBREGIONS.xlsx')

In [3]:
#Reading Users data
user_data  = pd.read_csv("users.csv", parse_dates=True)
user_data.sort_values(['created_at'],inplace= True)

#Convert created_at field to datetime object
user_data['created_at'] = pd.to_datetime(user_data['created_at'],format='%Y-%m-%d %H:%M:%S.%f',exact=False)
#add weeknumber coloumn based on created date field - Getting isocalendar week
user_data['weekno'] = user_data['created_at'].map(lambda x: x.isocalendar()[1])

#-------------Age Bins --------------------------------------------------------
user_data['date_of_birth'] = pd.to_datetime(user_data['date_of_birth'])
#add weeknumber coloumn based on created date field
currentdt =  datetime.now()
#currentdt.date()
#user_data['age'] = relativedelta(currentdt ,user_data['date_of_birth']).years
user_data['age'] =  ((currentdt.date() - user_data['date_of_birth']).dt.days/365)
user_data['age'] = user_data.age.astype("int")
user_data.sort_values(['age'],inplace= True)
no_of_bins  =( max(user_data['age']) - min(user_data['age']))/7
bins =  [18,25,32,39,46,53,60,67,74,81,88]
group_names = ['18-25','25-32','32-39','39-46','46-53','53-60','60-67','67-74','74-81','81-88']
user_data['range'] = pd.cut(user_data['age'],bins=bins,labels=group_names,include_lowest= True)


#-------------Regions------------------------------------------------------------
user_data['regionName'] =  user_data['state'].map(lambda StateCode: regions[regions['State Code'] == StateCode].Region.to_string().split()[1])
user_data = user_data.rename(columns=({ 'id' : 'user_id'}))
user_data.regionName.replace(to_replace=')', value='Other',inplace=True)



#Renaming id to user_id for further use
user_data = user_data.rename(columns=({ 'id' : 'user_id'}))


In [4]:
#user_data.head(5)

tableuser = ff.create_table(user_data.iloc[0:10,[0,2,4,8,9,10,11] ])
plotly.offline.iplot(tableuser)


In [5]:
#Reading auto stash transfers 
transfers_data =  pd.read_csv("auto_stash_transfers.csv")
#Converting created_at date to Datetime - Date object
transfers_data['created_at'] = pd.to_datetime(transfers_data['created_at']).dt.date
#transfers_data.head(5)
table = ff.create_table(transfers_data[0:20])
plotly.offline.iplot(table)


In [6]:
#Created a dataset merging both the user and transfers information
merged = pd.merge(transfers_data,user_data, on='user_id', how='left')
tableuser = ff.create_table(merged.iloc[0:5,[1,2,5,7,11,12,13,14] ])
plotly.offline.iplot(tableuser)

In [7]:
#Getting list of users who made transfers without duplicates
unique_userids = transfers_data.user_id.unique()

#Number of users who made transfers
user_made_trasnfers = len(unique_userids)

#Number of users registered
userids = len(user_data.user_id.unique())

#Question no 1
Percent_of_users =  float(user_made_trasnfers*100)/userids

## Q1. What percentage of users have made an Auto-Stash transfer?

In [8]:
print(Percent_of_users)

19.3277004216


# For Question 2,4 and 5 i have combined the subsections together. So they follow the order 2.a,4.a,5.a,2.b,4.b,5.b and so on.

In [9]:
#Question no 2 
#Question no a

#Using pandasql to get number of cohorts and the corresponding number of users in each cohort from Total list of user registered
number_of_cohorts_week = pysqldf("select distinct(weekno) as 'cohorts',count(user_id) as 'no_of_users' from user_data group by weekno")
number_of_cohorts_week
#Using pandasql to get number of cohorts and the corresponding number of users in each cohort based on users who made transfers
usertransfers_per_cohort = pysqldf("select distinct(weekno) as 'cohorts',count(distinct(user_id)) as 'no_user_trns' from merged group by weekno")
usertransfers_per_cohort

#Merged the above data to get comparison of total no of users vs no of users who made transaction per cohort
total_share = pd.merge(number_of_cohorts_week,usertransfers_per_cohort,on='cohorts',how="left")
#Created column which shows percentage of no of users who made transfers based on total users in that cohort
total_share['percent_of_each_cohort'] = (total_share['no_user_trns']*100)/total_share['no_of_users']

#print(total_share)

# Q2.a. What percentage of each cohort has made an Auto-Stash transfer?

In [10]:

table_tot_share_week = ff.create_table(total_share)
plotly.offline.iplot(table_tot_share_week)

In [11]:
#g = sns.pairplot(total_share, size=3,vars=["cohorts", "no_of_users"])

#sns.jointplot(x="cohorts", y="no_of_users", data=total_share);

#sns.pairplot(total_share);

totalusers  = Bar(x=total_share['cohorts'],y=total_share['no_of_users'],name='',marker=dict(color='#ffcdd2'))

totaltrans  = Bar(x=total_share['cohorts'],y=total_share['no_user_trns'],name='',marker=dict(color='#A2D5F2'))

data= [totalusers,totaltrans]

layout = Layout(title="Number of users who made transfers vs Total number of Users",
                xaxis=dict(title='Cohorts by Week'),
                yaxis=dict(title='Users/Transaction'))
fig = Figure(data=data, layout=layout)

plotly.offline.iplot(fig)

In [12]:
#Question no 4
#Question no a

#Using pandasql to get number of cohorts and the corresponding number of users in each cohort from Total list of user registered
number_of_cohorts_range = pysqldf("select distinct(range) as 'cohorts',count(user_id) as 'no_of_users' from user_data group by range")
number_of_cohorts_range
#Using pandasql to get number of cohorts and the corresponding number of users in each cohort based on users who made transfers
usertransfers_per_cohort_range = pysqldf("select distinct(range) as 'cohorts',count(distinct(user_id)) as 'no_user_trns' from merged group by range")
usertransfers_per_cohort_range

#Merged the above data to get comparison of total no of users vs no of users who made transaction per cohort
total_share_range = pd.merge(number_of_cohorts_range,usertransfers_per_cohort_range,on='cohorts',how="left")
#Created column which shows percentage of no of users who made transfers based on total users in that cohort
total_share_range['percent_of_each_cohort'] = (total_share_range['no_user_trns']*100)/total_share_range['no_of_users']



# 4.a. What percentage of each cohort has made an Auto-Stash transfer?

In [13]:

table_tot_share_range = ff.create_table(total_share_range)
plotly.offline.iplot(table_tot_share_range)

In [14]:
totalusers  = Bar(x=total_share_range['cohorts'],y=total_share_range['no_of_users'],name='',marker=dict(color='#ffcdd2'))

totaltrans  = Bar(x=total_share_range['cohorts'],y=total_share_range['no_user_trns'],name='',marker=dict(color='#A2D5F2'))

percentusers  = Bar(x=total_share_range['cohorts'],y=total_share_range['percent_of_each_cohort'],name='',marker=dict(color='#A2D5F2'))

data= [totalusers,totaltrans,percentusers]

layout = Layout(title="Number of users who made transfers vs Total number of Users",
                xaxis=dict(title='Cohorts by Range'),
                yaxis=dict(title='Users/Transaction'))
fig = Figure(data=data, layout=layout)

plotly.offline.iplot(fig)

In [15]:
#Question no 5
#Question no a

#Using pandasql to get number of cohorts and the corresponding number of users in each cohort from Total list of user registered
number_of_cohorts_region = pysqldf("select distinct(regionName) as 'cohorts',count(user_id) as 'no_of_users' from user_data group by regionName")
number_of_cohorts_region
#Using pandasql to get number of cohorts and the corresponding number of users in each cohort based on users who made transfers
usertransfers_per_cohort_region = pysqldf("select distinct(regionName) as 'cohorts',count(distinct(user_id)) as 'no_user_trns' from merged group by regionName")
usertransfers_per_cohort_region

#Merged the above data to get comparison of total no of users vs no of users who made transaction per cohort
total_share_region = pd.merge(number_of_cohorts_region,usertransfers_per_cohort_region,on='cohorts',how="left")
#Created column which shows percentage of no of users who made transfers based on total users in that cohort
total_share_region['percent_of_each_cohort'] = (total_share_region['no_user_trns']*100)/total_share_region['no_of_users']

#print('5.a. What percentage of each cohort has made an Auto-Stash transfer?')
#print(total_share_region)


# 5.a. What percentage of each cohort has made an Auto-Stash transfer?

In [16]:

table_tot_share_region = ff.create_table(total_share_region)
plotly.offline.iplot(table_tot_share_region)

In [17]:
totalusers  = Bar(x=total_share_region['cohorts'],y=total_share_region['no_of_users'],name='',marker=dict(color='#ffcdd2'))

totaltrans  = Bar(x=total_share_region['cohorts'],y=total_share_region['no_user_trns'],name='',marker=dict(color='#A2D5F2'))

data= [totalusers,totaltrans]

layout = Layout(title="Number of users who made transfers vs Total number of Users",
                xaxis=dict(title='Cohorts by Region'),
                yaxis=dict(title='Users/Transaction'))
fig = Figure(data=data, layout=layout)

plotly.offline.iplot(fig)

In [18]:
#Question 2b

#Solution 1 - Looking at data at cohort level
cohorttotal=  pysqldf('''select distinct(weekno) as "cohorts",count(*) as "tot_no_of_transfers", count(distinct(user_id)) as "no_of_users" 
                      ,(count(*)/ count(distinct(user_id))) as "Average Transfers" from merged group by weekno''')
#cohorttotal.head(5)


# 2.b For users who have made an Auto-Stash transfer, what is the average number of transfers made?

In [19]:
#print(cohorttotal)
table_tot_share_week = ff.create_table(cohorttotal)
plotly.offline.iplot(table_tot_share_week)

In [20]:
totalusers  = Bar(x=cohorttotal['cohorts'],y=cohorttotal['no_of_users'],name='',marker=dict(color='#ffcdd2'))

totaltrans  = Bar(x=cohorttotal['cohorts'],y=cohorttotal['Average Transfers'],name='',marker=dict(color='#A2D5F2'))

data= [totaltrans]

layout = Layout(title="Average Number of transfers per Cohort",
                xaxis=dict(title='Cohorts by Region'),
                yaxis=dict(title='Users/Transaction'))
fig = Figure(data=data, layout=layout)

plotly.offline.iplot(fig)

In [21]:
#Question 4b

#Solution 1 - Looking at data at cohort level
cohorttotal=  pysqldf('''select distinct(range) as "cohorts",count(*) as "tot_no_of_transfers", count(distinct(user_id)) as "no_of_users" 
                      ,(count(*)/ count(distinct(user_id))) as "Average Transfers" from merged group by range''')
#cohorttotal.head(5)



# 4.b For users who have made an Auto-Stash transfer, what is the average number of transfers made?

In [22]:
table_tot_share_range = ff.create_table(cohorttotal)
plotly.offline.iplot(table_tot_share_range)
#print(cohorttotal)


In [23]:
totalusers  = Bar(x=cohorttotal['cohorts'],y=cohorttotal['no_of_users'],name='',marker=dict(color='#ffcdd2'))

totaltrans  = Bar(x=cohorttotal['cohorts'],y=cohorttotal['Average Transfers'],name='',marker=dict(color='#A2D5F2'))

data= [totaltrans]

layout = Layout(title="Average Number of transfers per Cohort",
                xaxis=dict(title='Cohorts by Range'),
                yaxis=dict(title='Transaction'))
fig = Figure(data=data, layout=layout)

plotly.offline.iplot(fig)

In [24]:
#Question 5b

#Solution 1 - Looking at data at cohort level
cohorttotal=  pysqldf('''select distinct(regionName) as "cohorts",count(*) as "tot_no_of_transfers", count(distinct(user_id)) as "no_of_users" 
                      ,(count(*)/ count(distinct(user_id))) as "Average Transfers" from merged group by regionName''')
#cohorttotal.head(5)



# 5.b For users who have made an Auto-Stash transfer, what is the average number of transfers made?

In [25]:
#print(cohorttotal)

table_tot_share_region = ff.create_table(cohorttotal)
plotly.offline.iplot(table_tot_share_region)

In [26]:
totalusers  = Bar(x=cohorttotal['cohorts'],y=cohorttotal['no_of_users'],name='',marker=dict(color='#ffcdd2'))

totaltrans  = Bar(x=cohorttotal['cohorts'],y=cohorttotal['Average Transfers'],name='',marker=dict(color='#A2D5F2'))

data= [totaltrans]

layout = Layout(title="Average Number of transfers per Cohort",
                xaxis=dict(title='Cohorts by Region'),
                yaxis=dict(title='Transaction'))
fig = Figure(data=data, layout=layout)

plotly.offline.iplot(fig)

In [27]:

avg_transaction_count =( float(len(merged))/float(len(merged.user_id.unique())))


**Question 2,4,5**
### Solution 2 -  Looking at entire data as whole
### 2.b For users who have made an Auto-Stash transfer, what is the average number of transfers made?



In [28]:
print(avg_transaction_count)

9.99952790105


In [29]:
#Adding columns for difference between days for transactions by user and difference in amount between transactions
merged.sort_values(['user_id','created_at_x'],inplace= True)
merged['daysdiff'] = merged.groupby(['user_id'])['created_at_x'].diff().apply(pd.Timedelta)
merged['amountdiff'] = merged.groupby(['user_id'])['amount'].diff()
merged['daysdiff'].fillna(0,inplace= True)
merged['amountdiff'].fillna(0,inplace= True)

tableuser = ff.create_table(merged.iloc[0:5,[1,11,12,13,14,15,16] ])
plotly.offline.iplot(tableuser)

In [30]:
#Aggregate data for every user by getting total number of days between transactions
avg_days_diff = pysqldf("select user_id, weekno,((sum(daysdiff)/86400000000000.00)/count(user_id)) as 'Days'   from merged group by user_id")
#avg_days_diff.head(5)
#aggregating days based on weekno and then calculating average no of days between transfers using dataset created in previous step
avg_days_diff_cohort = pysqldf("select weekno, (sum(days)/count(weekno)) as 'average frequency of transfers in days'  from avg_days_diff group by weekno")


C:\Anaconda2\lib\site-packages\pandasql\sqldf.py:58: UserWarning:

the 'timedelta' type is not supported, and will be written as integer values (ns frequency) to the database.



# 2.c For users who have made an Auto-Stash transfer, what is the average frequency of transfers in days?


In [31]:


table_tot_share_week = ff.create_table(avg_days_diff_cohort)
plotly.offline.iplot(table_tot_share_week)

In [32]:
#totalusers  = Bar(x=cohorttotal['cohorts'],y=cohorttotal['no_of_users'],name='',marker=dict(color='#ffcdd2'))

totaltrans  = Bar(x=avg_days_diff_cohort['weekno'],y=avg_days_diff_cohort['average frequency of transfers in days'],name='',marker=dict(color='#A2D5F2'))

data= [totaltrans]

layout = Layout(title="Average frequency of transfers per Cohort",
                xaxis=dict(title='Cohorts by Weekno'),
                yaxis=dict(title='Transaction'))
fig = Figure(data=data, layout=layout)

plotly.offline.iplot(fig)

In [33]:
#Aggregate data for every user by getting total number of days between transactions
avg_days_diff = pysqldf("select user_id, range,((sum(daysdiff)/86400000000000.00)/count(user_id)) as 'Days' from merged group by user_id")
#avg_days_diff.head(5)
#aggregating days based on range and then calculating average no of days between transfers using dataset created in previous step
avg_days_diff_cohort = pysqldf("select range, (sum(days)/count(range)) as 'average frequency of transfers in days'  from avg_days_diff group by range")


# 4.c For users who have made an Auto-Stash transfer, what is the average frequency of transfers in days?

In [34]:

table_tot_share_week = ff.create_table(avg_days_diff_cohort)
plotly.offline.iplot(table_tot_share_week)

In [35]:
#totalusers  = Bar(x=cohorttotal['cohorts'],y=cohorttotal['no_of_users'],name='',marker=dict(color='#ffcdd2'))

totaltrans  = Bar(x=avg_days_diff_cohort['range'],y=avg_days_diff_cohort['average frequency of transfers in days'],name='',marker=dict(color='#A2D5F2'))

data= [totaltrans]

layout = Layout(title="Average frequency of transfers per Cohort",
                xaxis=dict(title='Cohorts by Range'),
                yaxis=dict(title='Transaction'))
fig = Figure(data=data, layout=layout)

plotly.offline.iplot(fig)

In [36]:
#Aggregate data for every user by getting total number of days between transactions
avg_days_diff = pysqldf("select user_id, regionName,((sum(daysdiff)/86400000000000.00)/count(user_id)) as 'Days' from merged group by user_id")
#avg_days_diff.head(5)
#aggregating days based on regionName and then calculating average no of days between transfers using dataset created in previous step
avg_days_diff_cohort = pysqldf("select regionName, (sum(days)/count(regionName)) as 'average frequency of transfers in days'  from avg_days_diff group by regionName")


# 5.c For users who have made an Auto-Stash transfer, what is the average frequency of transfers in days?


In [37]:
table_tot_share_week = ff.create_table(avg_days_diff_cohort)
plotly.offline.iplot(table_tot_share_week)

In [38]:
#totalusers  = Bar(x=cohorttotal['cohorts'],y=cohorttotal['no_of_users'],name='',marker=dict(color='#ffcdd2'))

totaltrans  = Bar(x=avg_days_diff_cohort['regionName'],y=avg_days_diff_cohort['average frequency of transfers in days'],name='',marker=dict(color='#A2D5F2'))

data= [totaltrans]

layout = Layout(title="Average frequency of transfers per Cohort",
                xaxis=dict(title='Cohorts by Region Name'),
                yaxis=dict(title='Transaction'))
fig = Figure(data=data, layout=layout)

plotly.offline.iplot(fig)

In [39]:
#First find difference in amount of money transferred between consecutive transaction for every user
avg_amt_diff = pysqldf("select user_id,weekno, (sum(amountdiff)/count(user_id)) as 'Avg_amount_diff'  from merged group by user_id")

#Find average amount of money of transferred aggregated by group
avg_amt_diff_cohort = pysqldf("select weekno, (sum(Avg_amount_diff)/count(weekno)) as 'average increase in dollar value'  from avg_amt_diff group by weekno")



# 2.d For users who have made an Auto-Stash transfer, what is the average increase in dollar value from one transfer to the next?


In [40]:
#print(avg_amt_diff_cohort)
table_tot_share_week = ff.create_table(avg_amt_diff_cohort)
plotly.offline.iplot(table_tot_share_week)

In [41]:
#totalusers  = Bar(x=cohorttotal['cohorts'],y=cohorttotal['no_of_users'],name='',marker=dict(color='#ffcdd2'))

totaltrans  = Bar(x=avg_amt_diff_cohort['weekno'],y=avg_amt_diff_cohort['average increase in dollar value'],name='',marker=dict(color='#A2D5F2'))

data= [totaltrans]

layout = Layout(title="Average Increase in dollar value from one transfer to next",
                xaxis=dict(title='Cohorts by WeekNo'),
                yaxis=dict(title='Dollar Value'))
fig = Figure(data=data, layout=layout)

plotly.offline.iplot(fig)

In [42]:
#First find difference in amount of money transferred between consecutive transaction for every user
avg_amt_diff = pysqldf("select user_id,range, (sum(amountdiff)/count(user_id)) as 'Avg_amount_diff'  from merged group by user_id")

#Find average amount of money of transferred aggregated by group
avg_amt_diff_cohort = pysqldf("select range, (sum(Avg_amount_diff)/count(range)) as 'average increase in dollar value'  from avg_amt_diff group by range")


# 4.d For users who have made an Auto-Stash transfer, what is the average increase in dollar value from one transfer to the next?

In [43]:
#print(avg_amt_diff_cohort)
table_tot_share_week = ff.create_table(avg_amt_diff_cohort)
plotly.offline.iplot(table_tot_share_week)

In [44]:
#totalusers  = Bar(x=cohorttotal['cohorts'],y=cohorttotal['no_of_users'],name='',marker=dict(color='#ffcdd2'))

totaltrans  = Bar(x=avg_amt_diff_cohort['range'],y=avg_amt_diff_cohort['average increase in dollar value'],name='',marker=dict(color='#A2D5F2'))

data= [totaltrans]

layout = Layout(title="Average Increase in dollar value from one transfer to next",
                xaxis=dict(title='Cohorts by Range'),
                yaxis=dict(title='Dollar Value'))
fig = Figure(data=data, layout=layout)

plotly.offline.iplot(fig)

In [45]:
#First find difference in amount of money transferred between consecutive transaction for every user
avg_amt_diff = pysqldf("select user_id,regionName, (sum(amountdiff)/count(user_id)) as 'Avg_amount_diff'  from merged group by user_id")

#Find average amount of money of transferred aggregated by group
avg_amt_diff_cohort = pysqldf("select regionName, (sum(Avg_amount_diff)/count(regionName)) as 'average increase in dollar value'  from avg_amt_diff group by regionName")


# 5.d For users who have made an Auto-Stash transfer, what is the average increase in dollar value from one transfer to the next?


In [46]:
#print(avg_amt_diff_cohort)
table_tot_share_week = ff.create_table(avg_amt_diff_cohort)
plotly.offline.iplot(table_tot_share_week)

In [47]:
#totalusers  = Bar(x=cohorttotal['cohorts'],y=cohorttotal['no_of_users'],name='',marker=dict(color='#ffcdd2'))

totaltrans  = Bar(x=avg_amt_diff_cohort['regionName'],y=avg_amt_diff_cohort['average increase in dollar value'],name='',marker=dict(color='#A2D5F2'))

data= [totaltrans]

layout = Layout(title="Average Increase in dollar value from one transfer to next",
                xaxis=dict(title='Cohorts by Region Name'),
                yaxis=dict(title='Dollar Value'))
fig = Figure(data=data, layout=layout)

plotly.offline.iplot(fig)

In [48]:
#Aggregate by userid differnce in days between transfers which have days difference greater than 30 days as per definition of churned users
churn_users_df = pysqldf("select weekno,range,regionName,risk_level, user_id, (daysdiff/86400000000000.00) as 'Days'  from merged where (daysdiff/86400000000000.00) >30 order by user_id")


# Q3. For each of the weekly cohorts from #2, what has been the Auto-Stash churn rate?

Detailed by all the cohorts by week, age range and region


**For Week based Cohorts**

In [49]:
churned_users_by_week =  pysqldf("select weekno, count(distinct(user_id)) as 'No Of Users Churned' from churn_users_df  group by weekno")
#churned_users_by_week
table_tot_share_week = ff.create_table(churned_users_by_week)
plotly.offline.iplot(table_tot_share_week)

In [50]:
#totalusers  = Bar(x=cohorttotal['cohorts'],y=cohorttotal['no_of_users'],name='',marker=dict(color='#ffcdd2'))

totaltrans  = Bar(x=churned_users_by_week['weekno'],y=churned_users_by_week['No Of Users Churned'],name='',marker=dict(color='#A2D5F2'))

data= [totaltrans]

layout = Layout(title="Churned Users per Cohort",
                xaxis=dict(title='Cohorts by Week No'),
                yaxis=dict(title='Dollar Value'))
fig = Figure(data=data, layout=layout)

plotly.offline.iplot(fig)

In [51]:
churned_users_by_week = churned_users_by_week.rename(columns=({ 'weekno' : 'cohorts'}))
churned_users_by_week
churned_users_by_week = pd.merge(number_of_cohorts_week,churned_users_by_week,on='cohorts',how="left")
churned_users_by_week['churn rate'] = (churned_users_by_week['No Of Users Churned']*100)/churned_users_by_week['no_of_users']
#print(churned_users_by_week)
table_tot_share_week = ff.create_table(churned_users_by_week)
plotly.offline.iplot(table_tot_share_week)

In [52]:
#totalusers  = Bar(x=cohorttotal['cohorts'],y=cohorttotal['no_of_users'],name='',marker=dict(color='#ffcdd2'))

totaltrans  = Bar(x=churned_users_by_week['cohorts'],y=churned_users_by_week['churn rate'],name='',marker=dict(color='#A2D5F2'))

data= [totaltrans]

layout = Layout(title="Churn Rate per Cohort",
                xaxis=dict(title='Cohorts by Week No'),
                yaxis=dict(title='Churn'))
fig = Figure(data=data, layout=layout)

plotly.offline.iplot(fig)

**For Age Range based Cohorts**



In [53]:
churned_users_by_range =  pysqldf("select range, count(distinct(user_id)) as 'No Of Users Churned' from churn_users_df  group by range")
#print(churned_users_by_week)
table_tot_share_week = ff.create_table(churned_users_by_range)
plotly.offline.iplot(table_tot_share_week)

In [54]:
#totalusers  = Bar(x=cohorttotal['cohorts'],y=cohorttotal['no_of_users'],name='',marker=dict(color='#ffcdd2'))

totaltrans  = Bar(x=churned_users_by_range['range'],y=churned_users_by_range['No Of Users Churned'],name='',marker=dict(color='#A2D5F2'))

data= [totaltrans]

layout = Layout(title="Churned Users per Cohort",
                xaxis=dict(title='Cohorts by Range'),
                yaxis=dict(title='Users'))
fig = Figure(data=data, layout=layout)

plotly.offline.iplot(fig)

In [55]:
churned_users_by_range = churned_users_by_range.rename(columns=({ 'range' : 'cohorts'}))
churned_users_by_range
churned_users_by_range = pd.merge(number_of_cohorts_range,churned_users_by_range,on='cohorts',how="left")
churned_users_by_range['churn rate'] = (churned_users_by_range['No Of Users Churned']*100)/churned_users_by_range['no_of_users']

#print(churned_users_by_range)
table_tot_share_week = ff.create_table(churned_users_by_range)
plotly.offline.iplot(table_tot_share_week)

In [56]:
#totalusers  = Bar(x=cohorttotal['cohorts'],y=cohorttotal['no_of_users'],name='',marker=dict(color='#ffcdd2'))

totaltrans  = Bar(x=churned_users_by_range['cohorts'],y=churned_users_by_range['churn rate'],name='',marker=dict(color='#A2D5F2'))

data= [totaltrans]

layout = Layout(title="Churn Rate per Cohort",
                xaxis=dict(title='Cohorts by Range'),
                yaxis=dict(title='Churn'))
fig = Figure(data=data, layout=layout)

plotly.offline.iplot(fig)

**For Region based Cohorts**

In [57]:
churned_users_by_region =  pysqldf("select regionName, count(distinct(user_id)) as 'No Of Users Churned' from churn_users_df  group by regionName")
 
#print(churned_users_by_week)
table_tot_share_week = ff.create_table(churned_users_by_region)
plotly.offline.iplot(table_tot_share_week)

In [58]:
#totalusers  = Bar(x=cohorttotal['cohorts'],y=cohorttotal['no_of_users'],name='',marker=dict(color='#ffcdd2'))

totaltrans  = Bar(x=churned_users_by_region['regionName'],y=churned_users_by_region['No Of Users Churned'],name='',marker=dict(color='#A2D5F2'))

data= [totaltrans]

layout = Layout(title="Churned Users per Cohort",
                xaxis=dict(title='Cohorts by Region'),
                yaxis=dict(title='Users'))
fig = Figure(data=data, layout=layout)

plotly.offline.iplot(fig)

In [59]:
churned_users_by_region = churned_users_by_region.rename(columns=({ 'regionName' : 'cohorts'}))
churned_users_by_region
churned_users_by_region = pd.merge(number_of_cohorts_region,churned_users_by_region,on='cohorts',how="left")
churned_users_by_region['churn rate'] = (churned_users_by_region['No Of Users Churned']*100)/churned_users_by_region['no_of_users']


table_tot_share_week = ff.create_table(churned_users_by_region)
plotly.offline.iplot(table_tot_share_week)

In [60]:
#totalusers  = Bar(x=cohorttotal['cohorts'],y=cohorttotal['no_of_users'],name='',marker=dict(color='#ffcdd2'))

totaltrans  = Bar(x=churned_users_by_region['cohorts'],y=churned_users_by_region['churn rate'],name='',marker=dict(color='#A2D5F2'))

data= [totaltrans]

layout = Layout(title="Churn Rate per Cohort",
                xaxis=dict(title='Cohorts by Region'),
                yaxis=dict(title='Churn'))
fig = Figure(data=data, layout=layout)

plotly.offline.iplot(fig)

# Question 6

What other insights can you gather given the data available?



**The insights generated are more generalized based on descriptive analysis of the data by aggregating information. The insights 
are not highly statistics oriented and intended for generic audience**

**The analysis focuses a lot on the Risk parameter and also some analysis with a drill down into State Level details**

In [61]:
users_by_risk  = pysqldf("select risk_level,count(distinct(user_id)) as'No Of Users Registered' from user_data group by risk_level")


table_tot_share_week = ff.create_table(users_by_risk)
plotly.offline.iplot(table_tot_share_week)

Based on the above table aggregated by Risk Level and the assumption that the Risk Level increases from 1(Low Level Risk),
2 (Medium Risk) and 3(High Risk).

**Seems like No of users who register are higher in Medium Risk category and fairly low on the High-Risk Category. **
**This seems a good indicator that people with appetite for Medium risk are more interested in using the Auto Stash Transfer**

Suggestion for Marketing Team
Possibly a good idea would be to try and incentivize lower risk customers (Risk Level 1) to capture a big user base because popularity of 
most Mobile apps is witnessed by higher user base

In [62]:
users_trnsf_by_risk  = pysqldf('''select risk_level,count(id) as"No Of Trans" ,count(distinct(user_id)) 
                                as "No Of Users Trns", (count(id)/count(distinct(user_id))) as "Avg Trans",
                                sum(amount) as "Tot Amt Trans",(sum(amount)/count(id)) as "Avg Trans Amt"
                                from merged group by risk_level''')

print("")
table_tot_share_week = ff.create_table(users_trnsf_by_risk)
plotly.offline.iplot(table_tot_share_week)

The above table aggregates information based on risk level by looking at metrics like
1. No of Transfers
2. No of Users who transffered 
3. Average number of transfers
4. Total Amount of Transfers
5. Average AMount of Transfer

The metrics play an important role in understanding frequency of transactions and dollar impact based on the risk level.
Below are depicted graphically some of the key figures.

In [63]:
#totalusers  = Bar(x=cohorttotal['cohorts'],y=cohorttotal['no_of_users'],name='',marker=dict(color='#ffcdd2'))

totaltrans  = Bar(x=users_trnsf_by_risk['risk_level'],y=users_trnsf_by_risk['Avg Trans'],name='',marker=dict(color='#A2D5F2'))

data= [totaltrans]

layout = Layout(title="Average transfers by Risk Level",
                xaxis=dict(title='Risk Level'),
                yaxis=dict(title='Transfers'))
fig = Figure(data=data, layout=layout)

plotly.offline.iplot(fig)

Average no of Transfers by Risk Level above depicts how the frequency of transaction for a lower user base in 
case High Risk (Level 3) is still higher as compared to Medium Risk (Level 2) and Low Risk (Level 1)

**This analysis probably provides an opportunity for revenue growth by charging for transactions that exceed 
certain limit for High Risk customers.**

**This decision could be made by looking the average transactions and some customers who are outliers would be charged. 
This would avoid the effect of cannibalization effect of customers moving from high risk to medium risk profile in 
order to avoid the charges on additional transactions.**


In [64]:
#totalusers  = Bar(x=cohorttotal['cohorts'],y=cohorttotal['no_of_users'],name='',marker=dict(color='#ffcdd2'))

totaltrans  = Bar(x=users_trnsf_by_risk['risk_level'],y=users_trnsf_by_risk['Avg Trans Amt'],name='',marker=dict(color='#A2D5F2'))

data= [totaltrans]

layout = Layout(title="Average Amount transfers by Risk Level",
                xaxis=dict(title='Risk Level'),
                yaxis=dict(title='Amount Transfer'))
fig = Figure(data=data, layout=layout)

plotly.offline.iplot(fig)

Average Amount of transfer based on risk level provides us with something that is expected that customer in High Risk profile 
have larger appetite and invest more money. 

**The difference if compared to volume of transactions seems striking since the volume transactions has 30% difference between
the risk level from prevous chart but amount of transactions between Medium Risk(2)/Low Risk(1) and High Risk (3) 
is more than 100%. **

In [66]:
churned_users_by_risk =  pysqldf('''select risk_level, count(distinct(user_id)) as 'No Of Users Churned' 
                                   from churn_users_df  group by risk_level''')
final = pd.merge(users_by_risk,churned_users_by_risk,on='risk_level')
final["Proportion"] =  final["No Of Users Churned"]/final["No Of Users Registered"]
table_tot_share_week = ff.create_table(final)
plotly.offline.iplot(table_tot_share_week)

In [67]:
users_trnsf_by_risk_region  = pysqldf('''select risk_level,regionName,count(id) as"No Of Trans" ,count(distinct(user_id)) 
                                as "No Of Users Trans", (count(id)/count(distinct(user_id))) as "Avg Trans",
                                (sum(amount)/count(id)) as "Avg Trans Amt"
                                from merged group by risk_level,regionName''')

table_tot_share_week = ff.create_table(users_trnsf_by_risk_region)
plotly.offline.iplot(table_tot_share_week)

The above table gives some information on metrics of No of Transfers, Average Transfers and Average Transfer Amount based on
the combination of Region and Risk.


In [68]:
risk = user_data['risk_level'].unique()

west = {
  'x': risk,
  'y': users_trnsf_by_risk_region[users_trnsf_by_risk_region['regionName']== 'West']['Avg Trans Amt'],
  'name': 'West',
  'type': 'bar'
};
south = {
  'x': risk,
  'y': users_trnsf_by_risk_region[users_trnsf_by_risk_region['regionName']== 'South']['Avg Trans Amt'],
  'name': 'South',
  'type': 'bar'
};
Northeast = {
  'x': risk,
  'y': users_trnsf_by_risk_region[users_trnsf_by_risk_region['regionName']== 'Northeast']['Avg Trans Amt'],
  'name': 'Northeast',
  'type': 'bar'
};
midwest = {
  'x': risk,
  'y': users_trnsf_by_risk_region[users_trnsf_by_risk_region['regionName']== 'Midwest']['Avg Trans Amt'],
  'name': 'Midwest',
  'type': 'bar'
};
Other = {
  'x': risk,
  'y': users_trnsf_by_risk_region[users_trnsf_by_risk_region['regionName']== 'Other']['Avg Trans Amt'],
  'name': 'Other',
  'type': 'bar'
};
 
data = [west, south, Northeast, midwest,Other];
layout = {
  'xaxis': {'title': 'X axis'},
  'yaxis': {'title': 'Y axis'},
  'barmode': 'relative',
  'title': 'Average Transfer Amount based on Risk Level and Region'
};
plotly.offline.iplot({'data': data, 'layout': layout})

**The above chart shows indicates how for Low Risk Level how the average transfer amount is mostly equally distributed across
the regions but incase of Medium Risk(2) the Other regions average amount transfer is fairly low.**

For High Risk seems like population from West Region have higher appetite and next is Southern Region but rest of the regions 
seem to be equally distributed

In [69]:
df= pysqldf('''select state, regionName,sum(amount) as "Total Amount", (sum(amount)/count(id)) as "Average Transfer Amount" from merged group by state ''')
table_tot_share_week = ff.create_table(df[0:10])
plotly.offline.iplot(table_tot_share_week)

The above table shows a glimpse of tabular information based on state the Total Amount of transfers and Average Transfer Amount

In [70]:

for col in df.columns:
    df[col] = df[col].astype(str)

scl = [[0.0, 'rgb(242,240,247)'],[0.2, 'rgb(218,218,235)'],[0.4, 'rgb(188,189,220)'],\
            [0.6, 'rgb(158,154,200)'],[0.8, 'rgb(117,107,177)'],[1.0, 'rgb(84,39,143)']]

#df['text'] = df['state'] + '<br>' +\
 #   'Region '+df['regionName'] + 'Average Amount ' + df['Average Transfer Amount']

    
df['text'] = 'State: ' + df['state'] + '<br>' +\
    'Region: '+df['regionName']+ '<br>' +\
    'Average Amount '+df['Average Transfer Amount']+'<br>'+\
    'Total Amount '+df['Total Amount']   
    
    
data = [ dict(
        type='choropleth',
        colorscale = scl,
        autocolorscale = False,
        locations = df['state'],
        z = df['Total Amount'].astype(float),
        locationmode = 'USA-states',
        text = df['text'],
        marker = dict(
            line = dict (
                color = 'rgb(255,255,255)',
                width = 2
            ) ),
        colorbar = dict(
            title = "$ USD")
        ) ]

layout = dict(
        title = 'Amount of money transferred by State',
        geo = dict(
            scope='usa',
            projection=dict( type='albers usa' ),
            showlakes = True,
            lakecolor = 'rgb(255, 255, 255)'),
             )
    
fig = dict( data=data, layout=layout )
plotly.offline.iplot( fig )

The Choropleth Map makes it easier to understand the concentration of amount of transferred based on the states in the USA.

**California seem to be the leading state in terms of investing money. THe probable cause of this could be more tech savvy population
reside in this state and make use of mobile apps to make investment.**

**The other states are Texas ad Florida that have high amount of moeny being transffered. This could also be attributed by general 
observation based on income levels and demographic information**

The more prominent regions seem to be the West and South of the USA.


In [71]:
df= pysqldf('''select state, regionName,count(user_id) as "Transfers", count(distinct(user_id)) as "Users" from merged group by state ''')
table_tot_share_week = ff.create_table(df[0:10])
plotly.offline.iplot(table_tot_share_week)

The table above gives similar information based on State wise distribution of Users and Transfers. The information is presented 
in Choropleth Map below.

In [72]:

for col in df.columns:
    df[col] = df[col].astype(str)

scl = [[0.0, 'rgb(242,240,247)'],[0.2, 'rgb(218,218,235)'],[0.4, 'rgb(188,189,220)'],\
            [0.6, 'rgb(158,154,200)'],[0.8, 'rgb(117,107,177)'],[1.0, 'rgb(84,39,143)']]

#df['text'] = df['state'] + '<br>' +\
 #   'Region '+df['regionName'] + 'Average Amount ' + df['Average Transfer Amount']

    
df['text'] = 'State: ' + df['state'] + '<br>' +\
    'Region: '+df['regionName']+ '<br>' +\
    'No of Transfers '+df['Transfers']+'<br>'+\
    'No of Users '+df['Users']   
    
    
data = [ dict(
        type='choropleth',
        colorscale = scl,
        autocolorscale = False,
        locations = df['state'],
        z = df['Transfers'],
        locationmode = 'USA-states',
        text = df['text'],
        marker = dict(
            line = dict (
                color = 'rgb(255,255,255)',
                width = 2
            ) ),
        colorbar = dict(
            title = "Count")
        ) ]

layout = dict(
        title = 'Number of Transfers by State',
        geo = dict(
            scope='usa',
            projection=dict( type='albers usa' ),
            showlakes = True,
            lakecolor = 'rgb(255, 255, 255)'),
             )
    
fig = dict( data=data, layout=layout )
plotly.offline.iplot( fig )

The Choropleth Map provides similar insight as the previous one reflecting higher number of transfers in state of California,
Texas and Florida.

**This could possibly mean there is high correlation between number of transfers and the amount of money transferred as well.**

**But it also reflects on the fact that lot of other states are probably under served or possible market regions that need
to be still captured.**

# What additional information would you like to incorporate if available?



**1. Additional information on demographics of users**
   1. Education Level
   2. Ethnicity 
   3. Marital Status
   4. Current Employment Status

**2. Additional Social Media Information**
   1. Analysis of Reviews of Users for the App on respective app store

   2. Users Tweets on Stash and Auto Stash Transfer
   
**3. Competitor information**




From current analysis and data standpoint, clarification on the following

1. Status of Users mean (Complete, Closed or In closing process)

2. Risk Level (Currently assumed)



# If you could recommend one thing to the marketing/product team what would that be?


**The one thing i would recommended would be build opportunity map to capture untapped user base **
**in other regions/States since lot of the States in Regions other than the West and South seem to be underserved.**




**Additionally:** 
Looking at some additional data especially Social Media Information. 
Performing Sentiment Analysis and Review analysis of Users would help gauge following things.

        a) This could help improve the product features and options available to invest or other aspects
        b) Understand or label users based on their reviews and build a model that would predict if they
             churned and apply to existing customers. By doing this Marketing team can probably know beforehand 
            who would churn and offer them some incentive to stay.
        
        Note: -The above suggestion was something that was presented as part of one of the capstone projects
               in NYU for fintech companies.
